In [21]:
import os
import time 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import json
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [22]:
BASE_PATH = "../data/processed"
TRAIN_PATHS = [
    "X_train_con_outliers.csv",
    "X_train_sin_outliers.csv",
]
TEST_PATHS = [
    "X_test_con_outliers.csv",
    "X_test_sin_outliers.csv"
]

# Leer los datasets de entrenamiento y prueba
TRAIN_DATASETS = [pd.read_csv(os.path.join(BASE_PATH, path)) for path in TRAIN_PATHS] # X_train 
TEST_DATASETS = [pd.read_csv(os.path.join(BASE_PATH, path)) for path in TEST_PATHS] # X_test

# Leer las etiquetas (target) de entrenamiento y prueba
y_train = pd.read_csv(os.path.join(BASE_PATH, "y_train.csv")).values.ravel()
y_test = pd.read_csv(os.path.join(BASE_PATH, "y_test.csv")).values.ravel()

In [23]:
all_selected_features = {}
model_results = []
k = 6

for index, dataset in enumerate(TRAIN_DATASETS):
    print(f"Procesando dataset {index + 1}")

    # Selección de características usando SelectKBest con f_classif (para clasificación)
    selector = SelectKBest(f_classif, k=k)
    X_train = selector.fit_transform(dataset, y_train)  # Ajuste y transformación para el conjunto de entrenamiento
    X_test = selector.transform(TEST_DATASETS[index])  # Transformación para el conjunto de prueba

    # Almacenar las características seleccionadas en el diccionario
    selected_features = dataset.columns[selector.get_support()].tolist()  # Obtener los nombres de las características seleccionadas
    all_selected_features[TRAIN_PATHS[index]] = selected_features  # Usar el nombre del archivo como clave

    ### ENTRENAR MODELO ###

    # Crear y entrenar el modelo de regresión logística (clasificación)
    model = AdaBoostClassifier(random_state = 42)  ### Asegúrate de ajustar los parámetros según sea necesario!!! 
    model.fit(X_train, y_train)

    # Predicciones en los datos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcular la precisión (accuracy) para los datos de entrenamiento
    accuracy_train = accuracy_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred, average='weighted')  # Asegúrate de ajustar el 'average'
    recall_train = recall_score(y_train, y_train_pred, average='weighted')
    f1_train = f1_score(y_train, y_train_pred, average='weighted')

    # Calcular la precisión (accuracy) para los datos de prueba
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred, average='weighted')
    recall_test = recall_score(y_test, y_test_pred, average='weighted')
    f1_test = f1_score(y_test, y_test_pred, average='weighted')

    # Almacenar los resultados en la lista, usando el nombre del archivo como "dataset"
    model_results.append(
    {
        "dataset_TRAIN": TRAIN_PATHS[index],  
        "dataset_TEST": TEST_PATHS[index],          
        "accuracy_train": accuracy_train,  
        "accuracy_test": accuracy_test,          
        "precision_train": precision_train, 
        "precision_test": precision_test,          
        "recall_train": recall_train,  
        "recall_test": recall_test,          
        "f1_train": f1_train,  
        "f1_test": f1_test,  
    }
)

model

Procesando dataset 1
Procesando dataset 2


AdaBoostClassifier(random_state=42)

In [24]:
### RESULTADOS 

df_results = pd.DataFrame(model_results)

# Guardar los resultados en un archivo JSON
with open(os.path.join(f"../models/model_results_k_{k}.json"), 'w') as json_file:
    json.dump(model_results, json_file, indent=4)

# Guardar las características seleccionadas en un solo archivo JSON
with open(os.path.join(f"../models/selected_features_k_{k}.json"), 'w') as json_file:
    json.dump(all_selected_features, json_file, indent=4)


### MUESTRA RESULTADOS ###

print(f"Resultados de todos los datasets con k: {k}:")
print(df_results.to_string(index=False))

best_dataset_index = df_results["accuracy_test"].idxmax()  
best_dataset = df_results.iloc[best_dataset_index]

print("\nEl mejor dataset seleccionado según la mayor precisión en el conjunto de prueba:")
print(best_dataset[["accuracy_train", "accuracy_test"]])


print(f"\nEl mejor dataset de entrenamiento es: {TRAIN_PATHS[best_dataset_index]}")
print(f"El mejor dataset de prueba es: {TEST_PATHS[best_dataset_index]}")

Resultados de todos los datasets con k: 6:
           dataset_TRAIN            dataset_TEST  accuracy_train  accuracy_test  precision_train  precision_test  recall_train  recall_test  f1_train  f1_test
X_train_con_outliers.csv X_test_con_outliers.csv        0.791531       0.759740         0.787318        0.758810      0.791531     0.759740  0.786038 0.759242
X_train_sin_outliers.csv X_test_sin_outliers.csv        0.807818       0.746753         0.804567        0.747813      0.807818     0.746753  0.803076 0.747251

El mejor dataset seleccionado según la mayor precisión en el conjunto de prueba:
accuracy_train    0.791531
accuracy_test      0.75974
Name: 0, dtype: object

El mejor dataset de entrenamiento es: X_train_con_outliers.csv
El mejor dataset de prueba es: X_test_con_outliers.csv


OPTMIZACIÓN

In [25]:
start_time = time.time()

# Crear y entrenar el modelo de regresión logística (clasificación)
model = AdaBoostClassifier(random_state=42)

# Definir el grid de hiperparámetros para AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],  # Aumentar el rango de estimadores
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Ampliar las opciones de tasa de aprendizaje
    'algorithm': ['SAMME'],  # Algoritmo para los estimadores 
}

# Configuración de GridSearchCV con validación cruzada estratificada
grid = GridSearchCV(
    model, 
    param_grid=param_grid, 
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),  # 5 pliegues
    n_jobs=-1,  # Utilizar todos los núcleos disponibles
    verbose=2,  # Nivel de detalle durante la ejecución
    scoring='accuracy',  # Evaluar el modelo según la precisión
    refit=True,  # Reajustar el modelo con los mejores parámetros encontrados
    return_train_score=False,  # No devolver puntuaciones de entrenamiento
    error_score='raise'  # Levantar error si alguna de las combinaciones falla
)

# Ajustar el modelo con el conjunto de entrenamiento filtrado
grid.fit(X_train, y_train)
final_model = grid.best_estimator_

# Predicciones en los datos de entrenamiento y prueba
y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)

end_time = time.time()
execution_time = end_time - start_time

data_to_save = {
    "hyperparams": grid.best_params_,
    "train_accuracy": accuracy_score(y_train, y_pred_train),
    "test_accuracy": accuracy_score(y_test, y_pred_test),
    "train_predictions": y_pred_train.tolist(),
    "test_predictions": y_pred_test.tolist(),
}

# Guardamos los resultados en un archivo JSON
with open(os.path.join("../models/hyperparams_Ada.json"), "w") as json_file:
    json.dump(data_to_save, json_file, indent=4)

print(f"Tiempo de ejecución: {execution_time / 60:.0f} minutos")
print(f"Training Accuracy: {accuracy_score(y_train, y_pred_train) * 100:.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_test) * 100:.4f}")

Fitting 10 folds for each of 20 candidates, totalling 200 fits


c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


Tiempo de ejecución: 1 minutos
Training Accuracy: 78.0130
Test Accuracy: 77.9221


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

acc = accuracy_score(y_test, y_pred_test)
prec = precision_score(y_test, y_pred_test)
rec = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
cm = confusion_matrix(y_test, y_pred_test)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

Accuracy: 0.7792207792207793
Precision: 0.7441860465116279
Recall: 0.5818181818181818
F1 Score: 0.6530612244897959
Confusion Matrix:
 [[88 11]
 [23 32]]


In [29]:
import pickle

# Guardamos el modelo en un archivo con pickle
with open("../models/AdaBoost_default_42.sav", "wb") as file:
    pickle.dump(model, file)